Construit 2 cartes : 
 - Top-5 destinations : villes où la météo est la plus agréable sur les 5 prochains jours (ici, températures les plus élevées) + affichage des conditions météo
 - Top-20 hotels pour chacune de ces 5 villes. Un lien permettra d'accéder à la page détail de chaque hôtel

In [1]:
import pandas as pd
import plotly.express as px
from variables import final_csv_name, weather_html_map, hotel_html_map

# Lecture du csv
df_final = pd.read_csv(final_csv_name)

# CARTE Météo TOP 5 des destinations
# On isole les colonnes utiles à la création de la carte
df_weather_map = df_final[["id_city", "city", "lat", "lon", "temperature", "condition"]].drop_duplicates()

# Top 5 des températures les plus élevées
df_weather_map = df_weather_map.sort_values("temperature", ascending = False).head(5)

# Arrondi la température (car on ne présente pas une température avec des virgules)
df_weather_map["temperature"] = df_weather_map["temperature"].apply(lambda x:round(x))

# Calcul la taille des bubbles : 
# La taille sera fonction de la température, sauf si celle-ci est <=0, dans ce cas on met la taille mini : 1
df_weather_map["size_temp"] = df_weather_map.apply(lambda x: 1 if x["temperature"] <= 0 else x["temperature"], axis = 1)

# Renommage des colonnes
df_weather_map = df_weather_map.rename({"temperature":"Température moyenne (°C)", "condition":"Conditions"}, axis = 1)

# Carte WEATHER
map_temp_moy = px.scatter_map(df_weather_map, lat = "lat", lon = "lon", 
                        size = "size_temp",
                        hover_name="city",
                        hover_data={"lat":False, "lon":False, "size_temp":False, "Conditions": True},
                        color = "Température moyenne (°C)",  color_continuous_scale = "Bluered",
                        zoom = 4, map_style = "open-street-map",
                        title = "Top-5 destinations sur les 5 prochains jours")

map_temp_moy.show()
map_temp_moy.write_html(weather_html_map)


# CARTE TOP 20 des hôtels pour le top-5 des villes ayant les températures les plus élevées
# On isole les colonnes utiles à la création de la carte
df_hotel_map = df_final.loc[df_final["id_city"].isin(df_weather_map["id_city"]), ["id_city", "city", "hotel_lat", "hotel_lon", "hotel_name", "hotel_score", "hotel_link"]]

# Lien hypertexte pour aller sur le détail de l'hotel
df_hotel_map = df_hotel_map.copy() # Créer une copie explicite pour éviter le warning
df_hotel_map["hotel_url"] = df_hotel_map.apply(lambda x: str("<a href=\'" + x["hotel_link"] + "'>" + x["hotel_name"] + "</a>"), axis=1) 

# Renommage des colonnes
df_hotel_map = df_hotel_map.rename({"hotel_score":"Note utilisateurs"}, axis=1)

# Carte HOTEL
map_hotel = px.scatter_map(df_hotel_map, lat = "hotel_lat", lon = "hotel_lon", 
                        hover_name="hotel_url",
                        hover_data={"city":False, "hotel_lat":False, "hotel_lon":False, "Note utilisateurs":True},
                        color="city",
                        zoom = 5, map_style = "open-street-map",
                        title = " Top-20 hôtels par ville")

# Taille des marqueurs
map_hotel.update_traces(marker={"size": 20, "opacity": 0.7})

# Couleur de fond du survol pour voir le lien hypertexte
map_hotel.update_layout(hoverlabel={"bgcolor":"white"})

map_hotel.show()
map_hotel.write_html(hotel_html_map)
